In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
os.chdir('/home/ubuntu/Notebooks/capstone2/src')
from training_data_cleaning import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import nltk
import warnings
import pickle

os.chdir('/home/ubuntu/Notebooks/data/ta')
ta_data = pd.read_pickle('ta_data2.pickle').replace(1.0,2.0).replace(0.0,2.0).sample(frac=.8,random_state=3)
ta_data = ta_data_cleaning(ta_data)
X = ta_data
X,y = balance_df_ta(X)


## LSTM

In [7]:
os.chdir('/home/ubuntu/Notebooks')
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz',
                                                 binary=True,
                                                 limit=3000000)

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer


X = ta_data['text']
y = ta_data['ratings_overall']
x_train,x_valid,y_train,y_valid = train_test_split(X,y,test_size=.15)

maxlen = 150
training_samples = x_train.shape[0]
validation_samples = x_valid.shape[0]
batch_size = 25
embedding_dims = 300
epochs = 4
embedding_dim = 300

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

token = Tokenizer(char_level=False,lower=True)
token.fit_on_texts(X)
# tokenize.texts_to_sequences(x_train.iloc[0])
x_train_seq = pad_sequences(token.texts_to_sequences(x_train),maxlen=maxlen)
x_valid_seq = pad_sequences(token.texts_to_sequences(x_valid),maxlen=maxlen)

word_index = token.word_index
max_words = len(word_index)

In [9]:
embedding_matrix = np.zeros((max_words,embedding_dims))
for word,i in word_index.items():
    if i < max_words:
        try:
            embedding_vector = word_vectors.get_vector(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        except:
            pass

In [10]:
(embedding_matrix.sum(axis=1) == 0).sum(),len(word_index)

(56582, 104758)

In [13]:
ohe = OneHotEncoder()
y_train_new = ohe.fit_transform(y_train.values.reshape(-1,1))
y_valid_new = ohe.fit_transform(y_valid.values.reshape(-1,1))

# x_valid_seq2 = x_valid_seq.reshape(x_valid_seq.shape[0],x_valid_seq.shape[1],1)
# x_train_seq2 = x_train_seq.reshape(x_train_seq.shape[0],x_train_seq.shape[1],1)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warn

In [64]:
x_valid_seq.shape

(7763, 150)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding

model = Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Dropout(rate=.2))
model.add(LSTM(batch_size,return_sequences=True))
model.add(Dropout(rate=.2))
model.add(LSTM(batch_size,return_sequences=False))
model.add(Dense(4,activation='softmax'))
model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',metrics=['acc'])
history = model.fit(x_train_seq,y_train_new.toarray(),epochs=epochs,
                     batch_size=batch_size,
                     validation_data=(x_valid_seq,
                                      y_valid_new))

Instructions for updating:
Use tf.cast instead.
Train on 131182 samples, validate on 23150 samples
Epoch 1/4
131182/131182 [==============================] - 2298s 18ms/step - loss: 0.8731 - acc: 0.6050 - val_loss: 0.7675 - val_acc: 0.6542
Epoch 2/4
131182/131182 [==============================] - 2297s 18ms/step - loss: 0.7210 - acc: 0.6808 - val_loss: 0.7535 - val_acc: 0.6613
Epoch 3/4
 52125/131182 [==========>...................] - ETA: 21:50 - loss: 0.6268 - acc: 0.7298

In [75]:
os.chdir('/home/ubuntu/Notebooks/data')

In [77]:
model2.save('first_model_ta.h5')

In [82]:
import pickle
f = open("word_dict_ta.pkl","wb")
pickle.dump(word_index,f)
f.close()